# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# Show all interactive result
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Ignore warning message
import warnings
warnings.simplefilter('ignore')

In [2]:
# import libraries
import pandas as pd
import os
from sqlalchemy import create_engine

from sklearn.utils import parallel_backend
import time

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('SELECT * FROM DisasterResponse', engine)
df.head(2)

# Use small set of data for quick pipeline test
df=df[0:2000]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
X = df['message'].values
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values
category_names = df.drop(['id', 'message', 'original', 'genre'], axis=1).columns.values

### 2. Write a tokenization function to process your text data

In [5]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /Users/RAYMOND/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/RAYMOND/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/RAYMOND/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/RAYMOND/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [6]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

def tokenize(text):
    # Normalization : Lower case and punctuation removal
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Word tokenization
    words = word_tokenize(text)
    
    # Remove Stop Words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Stemming
    stemmed = [PorterStemmer().stem(w) for w in words]

    return stemmed

#### Unit test for tokenize function

In [7]:
text = df.message[3]
print ('Before tokenize : ', text)
print ('After tokenize :', tokenize(text))

Before tokenize :  UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
After tokenize : ['un', 'report', 'leogan', '80', '90', 'destroy', 'hospit', 'st', 'croix', 'function', 'need', 'suppli', 'desper']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

def build_pipeline_1st():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier( RandomForestClassifier() ))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

#### Split data into train and test sets

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

#### Train pipeline (1st model)

In [10]:
start = time.time()

pipeline_1st = build_pipeline_1st()
pipeline_1st.fit(X_train, Y_train)

done = time.time()
print("Train Time = ", done - start, " sec")

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

Train Time =  9.771990776062012  sec


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Write classification report function for model performance evaluation

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

def evaluate_model(model, X_test, Y_test, category_names):    
    metric_names = ['ACCURACY', 'PRECISION', 'RECALL', 'F1']
    metric_df = pd.DataFrame(columns = metric_names)
    Y_pred = model.predict(X_test)
    
    all_accuracy, all_precision, all_recall, all_f1 = [], [], [], []
    for i, col in enumerate(category_names):
        y_true = Y_test[i]
        y_pred = Y_pred[i]
        
        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        metric_df = metric_df.append(pd.Series([acc, prec, recall, f1], index = metric_names, name = col)) 
    return metric_df

#### Evaluate model performance

In [12]:
metrics_1st = evaluate_model(pipeline_1st, X_test, Y_test, category_names)    

In [13]:
metrics_1st
metrics_1st.describe()

,ACCURACY,PRECISION,RECALL,F1
related,0.914286,1.000000,0.500000,0.666667
request,0.885714,0.000000,0.000000,0.000000
offer,0.971429,0.000000,0.000000,0.000000
aid_related,0.914286,1.000000,0.625000,0.769231
medical_help,0.942857,0.800000,0.800000,0.800000
medical_products,0.971429,0.000000,0.000000,0.000000
search_and_rescue,0.971429,0.000000,0.000000,0.000000
security,1.000000,0.000000,0.000000,0.000000
military,0.971429,0.000000,0.000000,0.000000
water,0.971429,0.000000,0.000000,0.000000


,ACCURACY,PRECISION,RECALL,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.937143,0.622857,0.409092,0.480856
std,0.052225,0.446228,0.336943,0.367692
min,0.800000,0.000000,0.000000,0.000000
25%,0.914286,0.000000,0.000000,0.000000
50%,0.942857,0.800000,0.500000,0.571429
75%,0.971429,1.000000,0.666667,0.800000
max,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
from sklearn.model_selection import GridSearchCV

def build_model_2nd():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier( RandomForestClassifier() ))
    ])
    
    parameters = {'vect__max_df' : [1.0, 0.8],
              'tfidf__use_idf' : [True, False],
              'clf__estimator__n_estimators' : [10, 20], 
              'clf__estimator__min_samples_split' : [2, 4]
    }
    
    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1)
    
    return cv

#### Build and train improved model (2nd model)

In [15]:
start = time.time()

with parallel_backend('multiprocessing'):
    model_2nd = build_model_2nd()
    model_2nd.fit(X_train, Y_train)

done = time.time()
print("Train Time = ", done - start, " sec")

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': [1.0, 0.8], 'tfidf__use_idf': [True, False], 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Train Time =  329.75347805023193  sec


In [16]:
model_2nd.best_params_

{'clf__estimator__min_samples_split': 2,
 'clf__estimator__n_estimators': 20,
 'tfidf__use_idf': True,
 'vect__max_df': 1.0}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

#### Evaluate model performance

In [17]:
metrics_2nd = evaluate_model(model_2nd, X_test, Y_test, category_names)    

In [18]:
metrics_2nd
metrics_2nd.describe()

,ACCURACY,PRECISION,RECALL,F1
related,0.942857,1.00,0.666667,0.800000
request,0.942857,1.00,0.333333,0.500000
offer,1.000000,0.00,0.000000,0.000000
aid_related,0.971429,1.00,0.875000,0.933333
medical_help,0.971429,1.00,0.800000,0.888889
medical_products,1.000000,0.00,0.000000,0.000000
search_and_rescue,0.971429,0.00,0.000000,0.000000
security,1.000000,0.00,0.000000,0.000000
military,0.942857,0.00,0.000000,0.000000
water,1.000000,1.00,1.000000,1.000000


,ACCURACY,PRECISION,RECALL,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.942041,0.692857,0.418786,0.503408
std,0.048252,0.450023,0.330411,0.361296
min,0.828571,0.000000,0.000000,0.000000
25%,0.942857,0.000000,0.000000,0.000000
50%,0.942857,1.000000,0.500000,0.666667
75%,0.971429,1.000000,0.690476,0.800000
max,1.000000,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### Write new feature extractor function "StartingVerbExtractor" to extract feature starting with verb

In [19]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if (pos_tags):   
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return 1    #True
        return 0    #False

    def fit(self, X, y=None):
        return self
    def test(self, text):
        print(test)
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [22]:
from sklearn.svm import SVC

def build_model_3rd():
    pipeline = Pipeline([
            ('features', FeatureUnion([
                ('text_pipeline', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ])),
                ('starting_verb', StartingVerbExtractor())
            ])),
            ('clf', MultiOutputClassifier( SVC() ))
        ])
    
    parameters = {'features__text_pipeline__vect__max_df' : [1.0],
                  'features__text_pipeline__tfidf__use_idf' : [True],
                  'clf__estimator__C' : [0.5, 1.0], 
                  'clf__estimator__kernel' : ['linear', 'rbf'],
                  'features__transformer_weights': (
                      {'text_pipeline': 1, 'starting_verb': 0.5},
                      {'text_pipeline': 0.5, 'starting_verb': 1},                      
                  )
    }
    
    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs = -1) # n_jobs = -1  for multiprocessing
    
    return cv

#### Build and train further improved model (3rd model)

In [23]:
start = time.time()

with parallel_backend('multiprocessing'):
    model_3rd = build_model_3rd()
    model_3rd.fit(X_train, Y_train)

done = time.time()
print("Train Time = ", done - start, " sec")    

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...ity=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'features__text_pipeline__vect__max_df': [1.0], 'features__text_pipeline__tfidf__use_idf': [True], 'clf__estimator__C': [0.5, 1.0], 'clf__estimator__kernel': ['linear', 'rbf'], 'features__transformer_weights': ({'text_pipeline': 1, 'starting_verb': 0.5}, {'text_pipeline': 0.5, 'starting_verb': 1})},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

Train Time =  358.94045901298523  sec


In [24]:
model_3rd.best_params_

{'clf__estimator__C': 1.0,
 'clf__estimator__kernel': 'linear',
 'features__text_pipeline__tfidf__use_idf': True,
 'features__text_pipeline__vect__max_df': 1.0,
 'features__transformer_weights': {'text_pipeline': 1, 'starting_verb': 0.5}}

#### Evaluate model performance

In [25]:
metrics_3rd = evaluate_model(model_3rd, X_test, Y_test, category_names)    

In [26]:
metrics_3rd
metrics_3rd.describe()

,ACCURACY,PRECISION,RECALL,F1
related,0.942857,1.000000,0.666667,0.800000
request,0.942857,0.666667,0.666667,0.666667
offer,1.000000,0.000000,0.000000,0.000000
aid_related,0.971429,1.000000,0.875000,0.933333
medical_help,0.971429,1.000000,0.800000,0.888889
medical_products,1.000000,0.000000,0.000000,0.000000
search_and_rescue,0.971429,0.000000,0.000000,0.000000
security,1.000000,0.000000,0.000000,0.000000
military,0.971429,0.000000,0.000000,0.000000
water,1.000000,1.000000,1.000000,1.000000


,ACCURACY,PRECISION,RECALL,F1
count,35.000000,35.000000,35.000000,35.000000
mean,0.950204,0.651905,0.469040,0.529539
std,0.047680,0.437495,0.355421,0.380456
min,0.828571,0.000000,0.000000,0.000000
25%,0.942857,0.000000,0.000000,0.000000
50%,0.971429,1.000000,0.625000,0.666667
75%,0.971429,1.000000,0.750000,0.857143
max,1.000000,1.000000,1.000000,1.000000


In [27]:
metrics_1st.mean()
metrics_2nd.mean()
metrics_3rd.mean()

ACCURACY     0.937143
PRECISION    0.622857
RECALL       0.409092
F1           0.480856
dtype: float64

ACCURACY     0.942041
PRECISION    0.692857
RECALL       0.418786
F1           0.503408
dtype: float64

ACCURACY     0.950204
PRECISION    0.651905
RECALL       0.469040
F1           0.529539
dtype: float64

Pick model with the largest F1 value as our best model

### 9. Export your model as a pickle file

In [28]:
import pickle

pickle.dump(model_3rd.best_estimator_, open('best_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.